In [0]:
import re
import pandas as pd
import json
import matplotlib.pyplot as plt



In [0]:
def null_check(df, column):
    # print(df[column])
    return df[column].isnull().sum()



In [0]:
def range_check(df, column, minimum, maximum):
    s = pd.Series(df[column].astype(str))
    # print(s)
    return len(s) - s.between(minimum, maximum).sum()



In [0]:
def total_error_percentage(df, errors):
    return (errors / df.size) * 100


In [0]:

def format_check(df, column, regex):
    non_errors = 0

    s = pd.Series(df[column]).astype(str)
    for string in s:
        if bool(re.fullmatch(regex, string)):
            non_errors += 1

    errors = len(s) - non_errors
    return errors

In [0]:
def value_check(df, column, value):
    errors = 0

    if isinstance(value, list):
        for row in df[column].astype(str):
            if row not in value:
                errors += 1

    else:
        for row in df[column].astype(str):
            if row != value:
                errors += 1

    return errors

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.appName("DataQuality").getOrCreate()

from pyspark.sql.types import StructType, StructField, FloatType, StringType
def visualize(data_dict, checks):
    # Define the schema
    schema = StructType([
        StructField("Columns", StringType(), True),
        StructField("Error_perc_for_"+checks, FloatType(), True)
    ])

    rows = []
    for key, value in data_dict.items():
        value = float(value) if isinstance(value, (float, int)) else value
        rows.append(Row(Columns=key, Null_percentage=value))

    df = spark.createDataFrame(rows, schema=schema)

    # Show the DataFrame
    display(df)
    return 

In [0]:

def run_quality_checks(dataset_index):
    result_map = {}

    with open('config.json', 'r') as file:
        errors = 0
        total_nulls = 0
        total_format_errors = 0
        total_value_errors = 0

        config = json.loads(file.read())

        dataset_profile = config[dataset_index]

        dataframe = pd.read_csv(dataset_profile['dataset'])

        # checks object
        checks = dataset_profile['checks'][0]

        # print(checks)

        # checks to run on the dataset
        keys = list(checks.keys())

        # print(keys)
        null_output = {}
        range_output = {}
        format_output = {}
        value_output = {}

        for i, check in enumerate(checks):

            # print(check)
            if check == 'null_check':

                for column in checks[check]:
                    # print(dataframe[column])
                    nulls = null_check(dataframe, column)

                    perc = (nulls / dataframe.size) * 100

                    null_output[column] = perc

                    total_nulls += nulls
                    errors += nulls
                    # print(nulls)


            elif check == "range_check":
                # columns
                if isinstance(checks[keys[i]], list):
                    for item in checks[keys[i]]:
                        for key, value in item.items():
                            column = key
                            range = value
                            constraints = range.split('-')
                            range_errors = range_check(dataframe, column, constraints[0], constraints[1])
                            range_output[column] = (range_errors / dataframe.size) * 100


                else:
                    columns = list(checks[check])
                    ranges = list(checks[check].values())
                    for i, column in enumerate(columns):
                        constraints = ranges[i].split('-')
                        # print(constraints[0], constraints[1])
                        range_errors = range_check(dataframe, column, constraints[0], constraints[1])
                        range_output[column] = (range_errors / dataframe.size) * 100



            elif check == "total_error_percentage":
                print("Total errors: ", total_error_percentage(dataframe, errors))

            elif check == "format_check":
                columns = list(checks[check].keys())
                regex = list(checks[check].values())
                for i, column in enumerate(columns):
                    print(column, regex[i])
                    format_errors = format_check(dataframe, column, regex[i])
                    errors += format_errors
                    total_format_errors += format_errors
                    format_output[column] = (format_errors / dataframe.size) * 100


            elif check == "value_check":
                columns = list(checks[check].keys())
                values = list(checks[check].values())
                for i, column in enumerate(columns):
                    value_errors = value_check(dataframe, column, values[i])
                    errors += value_errors
                    total_value_errors += value_errors
                    value_output[column] = (value_errors / dataframe.size) * 100


            elif check == "total_null_errors":
                print("Total nulls: ", total_nulls)

            elif check == "total_format_errors":
                print("Total format errors: ", total_format_errors)

            elif check == "total_value_errors":
                print("Total value errors: ", total_value_errors)

            elif check == "total_number_errors":
                print("Total errors: ", errors)

        visualize(null_output,"null_check")
        visualize(range_output,"range_check")
        visualize(format_output, "format_check")
        visualize(value_output, "value_check")
        
    return 



In [0]:

if __name__ == '__main__':
    for i in range(5):
        if i == 2:
            continue
        run_quality_checks(i)


    print("-"*100)


Total errors:  2.0238095238095237
Total errors:  17


Columns,Error_perc_for_null_check
Total energy consumption of primary fuels and equivalents,1.0714285
Fraction from renewable sources and waste,0.95238096


Databricks visualization. Run in Databricks to view.

Columns,Error_perc_for_range_check
Year,1.1904762


Databricks visualization. Run in Databricks to view.

...

Columns,Error_perc_for_range_check


Columns,Error_perc_for_format_check


Columns,Error_perc_for_value_check
INDICATOR,0.19132653
FREQ,0.44642857
TIME_FORMAT,0.19132653


Databricks visualization. Run in Databricks to view.

----------------------------------------------------------------------------------------------------
